# Join and preprocess multiple time series data

## Load data

In [1]:
path = '../../data/FRED/CORESTICKM159SFRBATL.csv'

In [17]:
import pandas as pd

df1 = pd.read_csv(filepath_or_buffer=path, index_col='observation_date', parse_dates=True)

In [18]:
df1.head(5)

,CORESTICKM159SFRBATL
observation_date,
1968-01-01,3.651861
1968-02-01,3.673819
1968-03-01,4.142164
1968-04-01,4.155828
1968-05-01,4.088245


In [19]:
df1.index

DatetimeIndex(['1968-01-01', '1968-02-01', '1968-03-01', '1968-04-01',
               '1968-05-01', '1968-06-01', '1968-07-01', '1968-08-01',
               '1968-09-01', '1968-10-01',
               ...
               '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01',
               '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01',
               '2025-11-01', '2025-12-01'],
              dtype='datetime64[ns]', name='observation_date', length=696, freq=None)

In [12]:
df1.plot()

In [20]:
path2 = '../../data/FRED/MORTGAGE30US.csv'
df2 = pd.read_csv(filepath_or_buffer=path2, index_col='observation_date',parse_dates=True)

df2.head(5)

,MORTGAGE30US
observation_date,
1971-04-02,7.33
1971-04-09,7.31
1971-04-16,7.31
1971-04-23,7.31
1971-04-30,7.29


In [9]:
df2.plot()

## Parse dates to join time series

In [ ]:
# as observation_date is the index in both dataframes and was parsed as datetime, we no longer need to sort by it
df = pd.concat(objs=[df1,df2],axis=1)
df.plot()

## Inner join vs outer join

In [24]:
# the default join on concat is outer 
df_outer = pd.concat(objs=[df1,df2],axis=1, join='outer') # which is implicitly the same as the above df

df_outer.plot()

In [25]:
# instead I actually want to use inner join 
df_inner = pd.concat(objs=[df1,df2],axis=1, join='inner')
df_inner.plot()

## Interpolate missing values

In [ ]:
# unfortunately the inner join removed values because the frequencies of inflation and mortgage rates are different

# look at what's currently in the dataframe 
pd.concat(objs=[df1,df2],axis=1).tail(50).style

# gaps in the observation dates are not regular!

,CORESTICKM159SFRBATL,MORTGAGE30US
observation_date,,
2025-04-01 00:00:00,3.176690,nan
2025-04-03 00:00:00,nan,6.640000
2025-04-10 00:00:00,nan,6.620000
2025-04-17 00:00:00,nan,6.830000
2025-04-24 00:00:00,nan,6.810000
2025-05-01 00:00:00,3.159131,6.760000
2025-05-08 00:00:00,nan,6.760000
2025-05-15 00:00:00,nan,6.810000
2025-05-22 00:00:00,nan,6.860000


In [28]:
# to maintain the full data we must interpolate the timeseries data 

df = df.interpolate(method = 'linear')

df.tail(20).style

,CORESTICKM159SFRBATL,MORTGAGE30US
observation_date,,
2025-09-25 00:00:00,3.140646,6.300000
2025-10-01 00:00:00,3.094288,6.320000
2025-10-02 00:00:00,3.070474,6.340000
2025-10-09 00:00:00,3.046660,6.300000
2025-10-16 00:00:00,3.022845,6.270000
2025-10-23 00:00:00,2.999031,6.190000
2025-10-30 00:00:00,2.975217,6.170000
2025-11-01 00:00:00,2.951403,6.195000
2025-11-06 00:00:00,2.962386,6.220000


In [29]:
df.plot()

## Iterate for seamless loading of multiple datasets

In [2]:
# in order to seamlessly loop over all datasets we create a list of paths
paths = [
    '../../data/FRED/CORESTICKM159SFRBATL.csv', # core inflation rate
    '../../data/FRED/MORTGAGE30US.csv', # 30 year mortgage rate
    '../../data/FRED/FEDFUNDS.csv', # federal funds rate
    '../../data/FRED/T10YIE.csv', # 10 year breakeven inflation rate
    '../../data/FRED/UNRATE.csv' # unemployment rate
]

# create an empty list to hold dataframes
dfs = []

# iterate over paths and save it into a list of dataframes
for path in paths:
    df = pd.read_csv(filepath_or_buffer=path, index_col='observation_date', parse_dates=True)
    dfs.append(df)

df = pd.concat(objs=dfs, axis=1).interpolate(method='linear')

In [3]:
df.tail(10).style

,CORESTICKM159SFRBATL,MORTGAGE30US,FEDFUNDS,T10YIE,UNRATE
observation_date,,,,,
2026-01-05 00:00:00,3.006318,6.155000,3.720000,2.260000,4.400000
2026-01-06 00:00:00,3.006318,6.156667,3.720000,2.270000,4.400000
2026-01-07 00:00:00,3.006318,6.158333,3.720000,2.270000,4.400000
2026-01-08 00:00:00,3.006318,6.160000,3.720000,2.270000,4.400000
2026-01-09 00:00:00,3.006318,6.140000,3.720000,2.280000,4.400000
2026-01-12 00:00:00,3.006318,6.120000,3.720000,2.290000,4.400000
2026-01-13 00:00:00,3.006318,6.100000,3.720000,2.300000,4.400000
2026-01-14 00:00:00,3.006318,6.080000,3.720000,2.290000,4.400000
2026-01-15 00:00:00,3.006318,6.060000,3.720000,2.290000,4.400000


In [4]:
df.plot()

## Export data

In [5]:
df.tail(10).style

,CORESTICKM159SFRBATL,MORTGAGE30US,FEDFUNDS,T10YIE,UNRATE
observation_date,,,,,
2026-01-05 00:00:00,3.006318,6.155000,3.720000,2.260000,4.400000
2026-01-06 00:00:00,3.006318,6.156667,3.720000,2.270000,4.400000
2026-01-07 00:00:00,3.006318,6.158333,3.720000,2.270000,4.400000
2026-01-08 00:00:00,3.006318,6.160000,3.720000,2.270000,4.400000
2026-01-09 00:00:00,3.006318,6.140000,3.720000,2.280000,4.400000
2026-01-12 00:00:00,3.006318,6.120000,3.720000,2.290000,4.400000
2026-01-13 00:00:00,3.006318,6.100000,3.720000,2.300000,4.400000
2026-01-14 00:00:00,3.006318,6.080000,3.720000,2.290000,4.400000
2026-01-15 00:00:00,3.006318,6.060000,3.720000,2.290000,4.400000


In [6]:
# change the name of the index 
df.index.name = 'date'

# rename the columns
df.rename(columns={
    'CORESTICKM159SFRBATL':'CPI',
    'MORTGAGE30US':'MR30',
    'FEDFUNDS':'FFR',
    'T10YIE':'INF10',
    'UNRATE':'UNRT'
})

# check the dataframe
df.tail(10).style

,CORESTICKM159SFRBATL,MORTGAGE30US,FEDFUNDS,T10YIE,UNRATE
date,,,,,
2026-01-05 00:00:00,3.006318,6.155000,3.720000,2.260000,4.400000
2026-01-06 00:00:00,3.006318,6.156667,3.720000,2.270000,4.400000
2026-01-07 00:00:00,3.006318,6.158333,3.720000,2.270000,4.400000
2026-01-08 00:00:00,3.006318,6.160000,3.720000,2.270000,4.400000
2026-01-09 00:00:00,3.006318,6.140000,3.720000,2.280000,4.400000
2026-01-12 00:00:00,3.006318,6.120000,3.720000,2.290000,4.400000
2026-01-13 00:00:00,3.006318,6.100000,3.720000,2.300000,4.400000
2026-01-14 00:00:00,3.006318,6.080000,3.720000,2.290000,4.400000
2026-01-15 00:00:00,3.006318,6.060000,3.720000,2.290000,4.400000


In [7]:
# write to excel 

df.to_excel('../../data/FRED/FRED_joined.xlsx', sheet_name='joined_data', index=True)